In [1]:
import pandas as pd
import numpy as np
from numpy import nan
import matplotlib.pyplot as plt
import seaborn as sns

#pd.set_option('display.max_columns', None)
df=pd.read_csv('Ravdess_TRAIN_transformed.csv')
df_test=pd.read_csv('Ravdess_TEST_transformed.csv')
df
#df scaled, var=0 and high correlated features removed, categorical features mapped

,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,sum,max,q25,...,stft_sum_w4,stft_std_w4,stft_q01_w4,stft_q05_w4,stft_q25_w4,stft_q50_w4,stft_q75_w4,stft_q95_w4,stft_kur_w4,stft_skew_w4
0,0,1,0,1,1,1,0,0.328567,-0.391762,0.342857,...,0.135483,-1.096766,-0.402429,1.421380,0.948188,0.831973,0.536448,0.0,10.826275,-2.710136
1,0,1,0,1,2,1,0,0.272964,-0.346159,0.342857,...,-0.196060,-1.013205,1.702463,1.092210,0.597843,0.441189,0.253557,0.0,0.064887,0.425032
2,0,1,0,0,1,1,0,0.337392,-0.284007,0.371429,...,-0.307833,-0.716367,-0.402429,0.899688,0.680414,0.652634,0.569396,0.0,3.519767,-1.290314
3,0,1,0,0,2,1,0,0.318693,-0.258631,0.342857,...,-0.607848,-0.803778,1.346476,0.835496,0.532143,0.515441,0.405521,0.0,0.218601,-0.015380
4,0,2,0,1,1,1,0,0.315162,-0.478371,0.342857,...,0.145139,-0.863050,1.018057,0.935801,0.644804,0.549313,0.296205,0.0,1.260594,-0.448574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1823,0,8,0,0,2,18,1,-0.212894,-0.426884,0.371429,...,-0.631100,0.407872,-0.402429,-0.568068,0.272635,0.320467,0.493310,0.0,1.457767,-1.346654
1824,0,8,1,1,1,18,1,-0.186141,-0.196663,0.342857,...,-0.887878,0.718169,-0.402429,-0.568068,-0.402428,0.139088,0.581452,0.0,-0.162612,-0.423159
1825,0,8,1,1,2,18,1,-0.314997,-0.216890,0.371429,...,-0.910208,0.339948,-0.402429,-0.225264,-0.114307,0.022347,0.470287,0.0,0.128201,-0.378094
1826,0,8,1,0,1,18,1,-0.088892,-0.214683,0.342857,...,-0.445016,0.715166,-0.402429,-0.568068,0.037070,0.207317,0.187007,0.0,0.356091,-0.924174


# Unbalancing emotion by considering only surprised emotion

In [3]:
emotion_demapper = {1:"neutral", 2:"calm", 3:"happy", 4:"sad", 5:"angry", 6:"fearful", 7:"disgust", 8:"surprised"}
df["emotion"] = df["emotion"].replace(emotion_demapper)
df_test["emotion"] = df_test["emotion"].replace(emotion_demapper)

emotion_mapper = {"neutral":0, "calm":0, "happy":0, "sad":0, "angry":0, "fearful":0, "disgust":0, "surprised":1}
df["emotion"] = df["emotion"].replace(emotion_mapper)
df_test["emotion"] = df_test["emotion"].replace(emotion_mapper)


# Partitioning

In [4]:
X, y = df.drop(["emotion", "vocal_channel", "emotional_intensity", "statement", "repetition", "actor", "sex"], axis=1), df[["emotion"]].values.flatten()
X2, y2 = df_test.drop(["emotion", "vocal_channel", "emotional_intensity", "statement", "repetition", "actor", "sex"], axis=1), df_test[["emotion"]].values.flatten()

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
#X_train, y_train = X, y
#X_test, y_test = X2, y2

# Imbalancing with KMeansSMOTE / SMOTE / SVMSMOTE

In [6]:
from collections import Counter

print('dataset shape %s' % Counter(y_train))

dataset shape Counter({0: 1178, 1: 101})


In [222]:
from imblearn.over_sampling import KMeansSMOTE, SMOTE, SVMSMOTE
from imblearn.over_sampling import RandomOverSampler

#ksmote = KMeansSMOTE(k_neighbors=8, n_jobs=-1, cluster_balance_threshold=0.1)
#smote = SMOTE()
svmsmote = SVMSMOTE()
#ros = RandomOverSampler()

X_resampled, y_resampled = svmsmote.fit_resample(X_train, y_train)


In [223]:
from collections import Counter

print('Resampled dataset shape %s' % Counter(y_resampled))

Resampled dataset shape Counter({1: 1178, 0: 1178})


# The effect of resampling on classifier result

# Classifier result on imbalanced data

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

clf = DecisionTreeClassifier(min_samples_leaf=10, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.9034608378870674
F1-score [0.9485936  0.20895522]
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       506
           1       0.29      0.16      0.21        43

    accuracy                           0.90       549
   macro avg       0.61      0.56      0.58       549
weighted avg       0.88      0.90      0.89       549



# Classifier result on balanced data

In [225]:
clf = DecisionTreeClassifier(min_samples_leaf=10, random_state=42)
clf.fit(X_resampled, y_resampled)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.9016393442622951
F1-score [0.94621514 0.42553191]
              precision    recall  f1-score   support

           0       0.95      0.94      0.95       506
           1       0.39      0.47      0.43        43

    accuracy                           0.90       549
   macro avg       0.67      0.70      0.69       549
weighted avg       0.91      0.90      0.91       549



# Imbalancing with ADASYN

In [8]:
from imblearn.over_sampling import ADASYN

adasyn = ADASYN(random_state=42, sampling_strategy='minority', n_neighbors=8)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_resampled))

Resampled dataset shape Counter({0: 1178, 1: 1176})


In [9]:
clf = DecisionTreeClassifier(min_samples_leaf=10, random_state=42)
clf.fit(X_resampled, y_resampled)

y_pred = clf.predict(X_test)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.8888888888888888
F1-score [0.93807107 0.46017699]
              precision    recall  f1-score   support

           0       0.96      0.91      0.94       506
           1       0.37      0.60      0.46        43

    accuracy                           0.89       549
   macro avg       0.67      0.76      0.70       549
weighted avg       0.92      0.89      0.90       549



In [244]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'sampling_strategy': ['minority', 'not minority', 'auto'],
    'n_neighbors': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 50],
    'random_state': [42]
}

adasyn = ADASYN()

grid_search = GridSearchCV(estimator=adasyn, param_grid=param_grid, scoring='recall')

grid_search.fit(X_train, y_train)


import warnings
warnings.filterwarnings('ignore')

print(grid_search.best_params_)

{'n_neighbors': 3, 'random_state': 42, 'sampling_strategy': 'minority'}


### Adasyn return the best result... Applying the dimensionality reduction and check the improvement on decisonTree classifier results

In [ ]:
#KNN without dimensionality reduction


### UMAP

In [12]:
%%time
#testing it with different number of features reduced
import umap

manifold = umap.UMAP(n_components=30, n_neighbors=35, min_dist=0.2, n_jobs=-1).fit(X_resampled, y_resampled)
X_train_UMAP = manifold.transform(X_resampled)

from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_UMAP, y_resampled)
#clf = DecisionTreeClassifier(min_samples_leaf=10, random_state=42)
#clf.fit(X_train_UMAP, y_resampled)

X_test_UMAP = manifold.transform(X_test)

y_pred = knn.predict(X_test_UMAP)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred, digits=3))

Accuracy 0.6320582877959927
F1-score [0.75305623 0.27857143]
              precision    recall  f1-score   support

           0      0.987     0.609     0.753       506
           1      0.165     0.907     0.279        43

    accuracy                          0.632       549
   macro avg      0.576     0.758     0.516       549
weighted avg      0.923     0.632     0.716       549

CPU times: total: 46.9 s
Wall time: 26 s


C:\Users\linov\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### MDS

In [249]:
%%time
from sklearn.manifold import MDS

mds = MDS(n_components=200, n_jobs=-1)
X_train_mds = mds.fit_transform(X_resampled)

clf = DecisionTreeClassifier(min_samples_leaf=3, random_state=42)
clf.fit(X_train_mds, y_resampled)

X_test_mds = mds.fit_transform(X_test)

y_pred = clf.predict(X_test_mds)

print('Accuracy %s' % accuracy_score(y_test, y_pred))
print('F1-score %s' % f1_score(y_test, y_pred, average=None))
print(classification_report(y_test, y_pred))

Accuracy 0.6994535519125683
F1-score [0.81927711 0.10810811]
              precision    recall  f1-score   support

           0       0.92      0.74      0.82       506
           1       0.07      0.23      0.11        43

    accuracy                           0.70       549
   macro avg       0.49      0.49      0.46       549
weighted avg       0.85      0.70      0.76       549

CPU times: total: 1.34 s
Wall time: 2min 44s
